# Inference using a CWL CommandLineTool

## Goal

Wrap the `inference` as a Common Workflow Language `CommandLineTool` and execute it with a CWL runner.

This notebook is linked to: https://eoap.github.io/inference-eoap/cwl-cli/inference/

## Dependencies

This notebook uses the output of the execution of the steps described in https://eoap.github.io/inference-eoap/cwl-workflow/stage-and-cog/

## Kernel

This noteboook uses the `bash` kernel.

## Setup

In [1]:
export WORKSPACE=/workspace/inference-eoap
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

rm -fr catalog.json
rm -fr S2A_T33TVM_20240125T100359_L1C-classification



## Run the inference 

Inspect and use `cwltool` to run the CommandLineTool definition:

In [2]:
cat ${WORKSPACE}/cwl-cli/inference.cwl | yq e -

cwlVersion: v1.0
class: CommandLineTool
id: inference
requirements:
  InlineJavascriptRequirement: {}
  EnvVarRequirement:
    envDef:
      PATH: /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
      PYTHONPATH: /app
  ResourceRequirement:
    coresMax: 2
    ramMax: 4096
hints:
  DockerRequirement:
    dockerPull: localhost/inference:latest
baseCommand: ["python", "-m", "app"]
arguments: []
inputs:
  input-item:
    type: Directory
    inputBinding:
      prefix: --input-item
outputs:
  stac_catalog:
    outputBinding:
      glob: .
    type: Directory


In [3]:
# check if the results.json file exists
if [ ! -f ${WORKSPACE}/runs/results.json ]; then
    echo "results.json file not found, run the stage-and-cog.cwl workflow first."
    exit 1
fi

cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-cli/inference.cwl \
    --input-item \
    $( cat ${WORKSPACE}/runs/results.json | jq -r .stac_catalog.path )

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved '/workspace/inference-eoap/cwl-cli/inference.cwl' to 'file:///workspace/inference-eoap/cwl-cli/inference.cwl'
WARNING [job inference] Skipping Docker software container '--memory' limit despite presence of ResourceRequirement with ramMin and/or ramMax setting. Consider running with --strict-memory-limit for increased portability assurance.
WARNING [job inference] Skipping Docker software container '--cpus' limit despite presence of ResourceRequirement with coresMin and/or coresMax setting. Consider running with --strict-cpu-limit for increased portability assurance.
INFO [job inference] /tmp/am8yu9it$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/am8yu9it,target=/rzXLuw \
    --mount=type=bind,source=/tmp/4bfy22zm,target=/tmp \
    --mount=type=bind,source=/workspace/inference-eoap/runs/ydmns4od,target=/var/lib/cwl/stgaba41d86-9e8c-4a3c-a932-1f0a182c9ab1/ydmns4od,readonly \
    --workdir